# 

In [1]:
from zipfile import ZipFile
from urllib.request import urlretrieve
import duckdb
import tempfile
import os

def extract_from_url(zipfile_url: str, filename: str, dir: str='.', overwrite: bool=False):
    filepath = os.path.join(dir, filename)
    if overwrite or not os.path.exists(filepath):
        with tempfile.NamedTemporaryFile() as zipped:
            urlretrieve(zipfile_url, zipped.name)
            with ZipFile(zipped) as zipfile:
                zipfile.extract(filename)

    return filepath

def create_pv_table(tsvfile: str, table_name: str, db: duckdb.DuckDBPyConnection, overwrite: bool=False):
    existing_tables = db.sql("show tables").df().name.values
    
    if overwrite or not table_name in existing_tables:
        table = db.read_csv(tsvfile, delimiter='\t', all_varchar=True)
        table.create(table_name)


Takes 4 seconds:

In [2]:
ddb = duckdb.connect('./patentsview.ddb')

g_patent_path = extract_from_url('https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip', 'g_patent.tsv')

csv_read_in = ddb.read_csv(g_patent_path, header=True, quotechar='"', delimiter="\t", all_varchar=True)
csv_read_in.write_parquet("g_patent.parquet")

parquet_read_in = ddb.read_parquet("g_patent.parquet")
parquet_read_in.create('g_patent')
ddb.table('g_patent')

┌───────────┬─────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬────────────┬───────────┬───────────────┐
│ patent_id │ patent_type │ patent_date │                                                                                                                 patent_title                                                                                                                  │ wipo_kind │ num_claims │ withdrawn │   filename    │
│  varchar  │   varchar   │   varchar   │                                                                                                                    varchar                                                                                                                    │  varchar  │  varchar   │  varchar  │    varchar 

Takes > 10 minutes (never finished):

In [ ]:
csv_read_in.create('g_patent_from_csv')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))